### **Run queries using DuckDB**

In [5]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

import os
import pandas as pd
import numpy as np
import duckdb as db
import altair as alt

#### Load data files saved from API

In [6]:
# collect parquet files
output_file = "../census_outputs/census_tract_2020.parquet/"

parq_files = [ os.path.join(output_file, file) for file in os.listdir(output_file) if ".parquet" in file ]

parq_files.sort()

In [7]:
# create DuckDB database
con = db.connect("census_tract.db")

In [8]:
# create table
for file in parq_files:
    if "census_tract" not in con.sql("SHOW TABLES;").df().name.tolist():
        con.execute(f"CREATE TABLE census_tract AS SELECT * FROM '{file}'")
        print("Created table census_tract")
        print(f"Inserted {file.split('/')[-1]} into census_tract table")

    else:
        con.execute(f"INSERT INTO census_tract SELECT * FROM '{file}'")
        print(f"Inserted {file.split('/')[-1]} into census_tract table")

Inserted part.0.parquet into census_tract table
Inserted part.1.parquet into census_tract table
Inserted part.10.parquet into census_tract table
Inserted part.100.parquet into census_tract table
Inserted part.101.parquet into census_tract table
Inserted part.102.parquet into census_tract table
Inserted part.103.parquet into census_tract table
Inserted part.104.parquet into census_tract table
Inserted part.105.parquet into census_tract table
Inserted part.106.parquet into census_tract table
Inserted part.107.parquet into census_tract table
Inserted part.108.parquet into census_tract table
Inserted part.109.parquet into census_tract table
Inserted part.11.parquet into census_tract table
Inserted part.110.parquet into census_tract table
Inserted part.111.parquet into census_tract table
Inserted part.112.parquet into census_tract table
Inserted part.113.parquet into census_tract table
Inserted part.114.parquet into census_tract table
Inserted part.115.parquet into census_tract table
Insert

In [9]:
# show row count
query = """
SELECT count(*)
FROM census_tract;
        """

con.sql(query).show()

┌──────────────┐
│ count_star() │
│    int64     │
├──────────────┤
│    223737546 │
└──────────────┘



In [10]:
# show row count
query = """
SELECT *
FROM census_tract
LIMIT 5;
        """

con.sql(query).show()

┌──────────────────────┬────────────┬────────────┬───┬──────────────────────┬────────────────────┬──────────────────┐
│        geo_id        │ state_code │ state_name │ … │     measurement      │ demographic_target │   demographic    │
│       varchar        │  varchar   │  varchar   │   │       varchar        │      varchar       │     varchar      │
├──────────────────────┼────────────┼────────────┼───┼──────────────────────┼────────────────────┼──────────────────┤
│ 1400000US01073003600 │ 01         │ Alabama    │ … │ estimate             │ households by type │ total households │
│ 1400000US01073003600 │ 01         │ Alabama    │ … │ annotation of esti…  │ households by type │ total households │
│ 1400000US01073003600 │ 01         │ Alabama    │ … │ margin of error      │ households by type │ total households │
│ 1400000US01073003600 │ 01         │ Alabama    │ … │ annotation of marg…  │ households by type │ total households │
│ 1400000US01073003600 │ 01         │ Alabama    │ … │ p

In [11]:
# create table of unique variables

query = """
DROP TABLE IF EXISTS variables;

CREATE TABLE variables AS 
SELECT variable_code, 
       variable_name, 
       measurement, 
       demographic_target, 
       demographic
FROM census_tract
GROUP BY 1, 2, 3, 4, 5
ORDER BY variable_code;
        """

con.execute(query)

print("Created table variables")
print()

# show first 10 rows
query = """
SELECT *
FROM variables
LIMIT 10;
        """

con.sql(query).show()

Created table variables

┌───────────────┬──────────────────────┬──────────────────────┬────────────────────┬───────────────────────────────────┐
│ variable_code │    variable_name     │     measurement      │ demographic_target │            demographic            │
│    varchar    │       varchar        │       varchar        │      varchar       │              varchar              │
├───────────────┼──────────────────────┼──────────────────────┼────────────────────┼───────────────────────────────────┤
│ DP02_0001E    │ Estimate!!HOUSEHOL…  │ estimate             │ households by type │ total households                  │
│ DP02_0001EA   │ Annotation of Esti…  │ annotation of esti…  │ households by type │ total households                  │
│ DP02_0001M    │ Margin of Error!!H…  │ margin of error      │ households by type │ total households                  │
│ DP02_0001MA   │ Annotation of Marg…  │ annotation of marg…  │ households by type │ total households                  │
│ DP02_

In [12]:
# create table of unique states

query = """
DROP TABLE IF EXISTS states;

CREATE TABLE states AS 
SELECT state_code, 
       state_name
FROM census_tract
GROUP BY 1, 2
ORDER BY state_code;
        """

con.execute(query)

print("Created table states")
print()

# show first 5 rows
query = """
SELECT *
FROM states
LIMIT 5;
        """

con.sql(query).show()

Created table states

┌────────────┬────────────┐
│ state_code │ state_name │
│  varchar   │  varchar   │
├────────────┼────────────┤
│ 01         │ Alabama    │
│ 02         │ Alaska     │
│ 04         │ Arizona    │
│ 05         │ Arkansas   │
│ 06         │ California │
└────────────┴────────────┘



In [13]:
# create table of unique tracts

query = """
DROP TABLE IF EXISTS tracts;

CREATE TABLE tracts AS 
SELECT geo_id, 
       tract_name, 
       tract, 
       county, 
       state_code 
FROM census_tract
GROUP BY 1, 2, 3, 4, 5
ORDER BY geo_id;
        """

con.execute(query)

print("Created table tracts")
print()

# show first 5 rows
query = """
SELECT *
FROM tracts
LIMIT 5;
        """

con.sql(query).show()

Created table tracts

┌──────────────────────┬──────────────────────────────────────────────┬─────────┬─────────┬────────────┐
│        geo_id        │                  tract_name                  │  tract  │ county  │ state_code │
│       varchar        │                   varchar                    │ varchar │ varchar │  varchar   │
├──────────────────────┼──────────────────────────────────────────────┼─────────┼─────────┼────────────┤
│ 1400000US01001020100 │ Census Tract 201, Autauga County, Alabama    │ 020100  │ 001     │ 01         │
│ 1400000US01001020200 │ Census Tract 202, Autauga County, Alabama    │ 020200  │ 001     │ 01         │
│ 1400000US01001020300 │ Census Tract 203, Autauga County, Alabama    │ 020300  │ 001     │ 01         │
│ 1400000US01001020400 │ Census Tract 204, Autauga County, Alabama    │ 020400  │ 001     │ 01         │
│ 1400000US01001020501 │ Census Tract 205.01, Autauga County, Alabama │ 020501  │ 001     │ 01         │
└──────────────────────┴─────────

#### Rank states by their average total forgeign born population average

In [14]:
def check_nulls(variable_code):
    # check for null rows for given given variable code
    query = f"""
    SELECT state_name, 
        variable_code, 
        variable_name, 
        value
    FROM census_tract
    WHERE variable_code = '{variable_code}'
            AND value IS NULL;
             """

    return con.sql(query)

def average_rank(variable_code):
    # rank by average value for given variable code
    query = f"""
    SELECT state_name, 
        variable_code,
        variable_name,
        ROUND(AVG(value::INT), 2) as avg_value,
        RANK() OVER (ORDER BY avg_value DESC NULLS LAST) as rank
    FROM census_tract
    WHERE variable_code = '{variable_code}'
    GROUP BY 1, 2, 3
    ORDER BY 5 asc;
             """

    return con.sql(query)

In [15]:
# check nulls for foreign born population estimate
check_nulls("DP02_0094E")

┌────────────┬───────────────┬───────────────┬─────────┐
│ state_name │ variable_code │ variable_name │  value  │
│  varchar   │    varchar    │    varchar    │ varchar │
├──────────────────────────────────────────────────────┤
│                        0 rows                        │
└──────────────────────────────────────────────────────┘

In [16]:
# rank stats by average total foreign born population estimate
average_rank("DP02_0094E").show()

┌───────────────┬───────────────┬──────────────────────────────────────────────────────────┬───────────┬───────┐
│  state_name   │ variable_code │                      variable_name                       │ avg_value │ rank  │
│    varchar    │    varchar    │                         varchar                          │  double   │ int64 │
├───────────────┼───────────────┼──────────────────────────────────────────────────────────┼───────────┼───────┤
│ California    │ DP02_0094E    │ Estimate!!PLACE OF BIRTH!!Total population!!Foreign born │   1155.79 │     1 │
│ New Jersey    │ DP02_0094E    │ Estimate!!PLACE OF BIRTH!!Total population!!Foreign born │    922.91 │     2 │
│ Florida       │ DP02_0094E    │ Estimate!!PLACE OF BIRTH!!Total population!!Foreign born │    853.59 │     3 │
│ New York      │ DP02_0094E    │ Estimate!!PLACE OF BIRTH!!Total population!!Foreign born │    808.01 │     4 │
│ Nevada        │ DP02_0094E    │ Estimate!!PLACE OF BIRTH!!Total population!!Foreign born │    

#### Compare foreign born ranking to the population of each language spoken

In [17]:
query = """
SELECT variable_code
FROM variables
WHERE demographic_target = 'ancestry'
        and measurement = 'estimate'
GROUP BY 1
        """

language_code_dict = con.sql(query).fetchnumpy()

language_codes = language_code_dict["variable_code"]

In [18]:
# check if any language codes have null
for language in language_codes:
    df = check_nulls(language).df()
    if len(df["state_name"]) > 0: 
        print("Null exists in:")
        print(df.loc[0, "state_name"])
        print("Null value count", len(df["state_name"]))

In [19]:
# add foreign born code to the language list so we can iterate at once
language_codes = np.append(language_codes, "DP02_0094E")

In [20]:
# collect all dataframe of all averagae ranked language codes
language_df_lst = [ average_rank(code).df() for code in language_codes ]

In [21]:
# combine all language dataframes
foreign_born_language = pd.concat(language_df_lst)

In [22]:
# check rankings for California (#1 ranked foreign born population)
foreign_born_language[foreign_born_language["state_name"] == "California"].sort_values("rank")

,state_name,variable_code,variable_name,avg_value,rank
0,California,DP02_0094E,Estimate!!PLACE OF BIRTH!!Total population!!Fo...,1155.79,1
1,California,DP02_0124E,Estimate!!ANCESTRY!!Total population,4328.69,2
5,California,DP02_0141E,Estimate!!ANCESTRY!!Total population!!Portuguese,37.00,6
5,California,DP02_0126E,Estimate!!ANCESTRY!!Total population!!Arab,34.80,6
12,California,DP02_0142E,Estimate!!ANCESTRY!!Total population!!Russian,40.33,13
17,California,DP02_0137E,Estimate!!ANCESTRY!!Total population!!Italian,154.40,18
17,California,DP02_0128E,Estimate!!ANCESTRY!!Total population!!Danish,16.73,18
17,California,DP02_0139E,Estimate!!ANCESTRY!!Total population!!Norwegian,38.41,18
17,California,DP02_0149E,Estimate!!ANCESTRY!!Total population!!Ukrainian,12.13,18
19,California,DP02_0151E,Estimate!!ANCESTRY!!Total population!!West Ind...,9.68,20


#### Create interactive chart for ancerstry population by state

In [23]:
# only return records from the ancestry demographic and that contains a country of origin
query = f"""
SELECT variable_code, REGEXP_REPLACE(demographic, 'total population ', '') as country
FROM variables
WHERE variable_code IN {tuple(foreign_born_language.variable_code.unique())}
 AND demographic_target = 'ancestry'
 AND SPLIT_PART(demographic, ' ', 3) != ''
         """

ancestry_country = con.sql(query).df()

In [24]:
ancestry_country.head()

,variable_code,country
0,DP02_0125E,american
1,DP02_0126E,arab
2,DP02_0127E,czech
3,DP02_0128E,danish
4,DP02_0129E,dutch


In [25]:
# join ancestry country to foreign_born_language df
foreign_born_language_wCountry = foreign_born_language.merge(ancestry_country, on = "variable_code", how = "inner")

In [26]:
foreign_born_language_wCountry = foreign_born_language_wCountry.rename({"avg_value": "avg_population_estimate"}, axis = 1)

In [27]:
foreign_born_language_wCountry.head()

,state_name,variable_code,variable_name,avg_population_estimate,rank,country
0,North Carolina,DP02_0143E,Estimate!!ANCESTRY!!Total population!!Scotch-I...,90.90,1,scotch-irish
1,South Carolina,DP02_0143E,Estimate!!ANCESTRY!!Total population!!Scotch-I...,86.20,2,scotch-irish
2,Tennessee,DP02_0143E,Estimate!!ANCESTRY!!Total population!!Scotch-I...,82.46,3,scotch-irish
3,West Virginia,DP02_0143E,Estimate!!ANCESTRY!!Total population!!Scotch-I...,59.41,4,scotch-irish
4,Virginia,DP02_0143E,Estimate!!ANCESTRY!!Total population!!Scotch-I...,55.76,5,scotch-irish


In [56]:
state_options = list(foreign_born_language_wCountry.state_name.unique())
state_options.sort()

input_dropdown = alt.binding_select(options = state_options)

dropdown_selection = alt.selection_single(fields = ["state_name"], bind = input_dropdown, name = "state")

ancestry_bar = alt.Chart(foreign_born_language_wCountry, title = "Average population estimate of ancestral homeland").mark_bar().encode(
    x=alt.X("country:N"), 
    y=alt.Y("avg_population_estimate:Q"),
    color=alt.Color('state_name:N')
).configure_title(fontSize=21
).properties(
    width=800,
    height=500
).configure_axis(
    titleFontSize=15
).add_selection(
    dropdown_selection
).transform_filter(
    dropdown_selection
)

/opt/homebrew/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'selection_single' is deprecated.  Use 'selection_point'
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)
/opt/homebrew/lib/python3.10/site-packages/altair/utils/deprecation.py:65: AltairDeprecationWarning: 'add_selection' is deprecated. Use 'add_params' instead.
  warnings.warn(message, AltairDeprecationWarning, stacklevel=1)


In [57]:
ancestry_bar

alt.Chart(...)